In [1]:
import threading
import serial
import serial.tools.list_ports
import pandas as pd
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

In [6]:
ports = serial.tools.list_ports.comports()
[(port.name, port.manufacturer) for port in ports]

[('COM1', '(Standard port types)')]

In [7]:
PORT='COM6'
BAUD=9600

p=[True,0]
plot_rate=0.5
plot_size=1000


In [8]:
ser = serial.Serial(port=PORT,baudrate=BAUD)

In [9]:
df=pd.DataFrame(columns=[])
df

""


In [10]:


def plot_data():
    p[0]=False
    clear_output(wait=True)
    #plt.figure(figsize=figsize)
    for label in df.columns:
        plt.plot(df.tail(plot_size).index,df.tail(plot_size)[label], label=label)
    #plt.title(title)
    #plt.grid(True)
    #plt.xlabel('epoch')
    #plt.legend(loc='center left') # the plot evolves to the right
    plt.show();
    p[0]=True

def recieve_data(name,value,type):
    #print(name,value,type)
    if type=="int":
        value=int(value)
    d = datetime.utcnow()
    s = (d - timedelta(microseconds=d.microsecond)) + timedelta(
        microseconds=int(d.microsecond/(1000*100))*(1000*100)
    )
    
    if name not in df.columns:
        df[name]=None
        
    if s in df.index:
        df.loc[s]={**df.loc[s],
                  name:value
                  }
        
    else:
        df.loc[s]={name:value}
        if p[0] and time.time()>p[1]+plot_rate:
            plot_data()
            p[1]=time.time()
    
    
    
        
def read_buff(buff):
    s=b"".join(buff).decode()
    s_parts=s.split(" ")
    ddic={}
    for p in s_parts[1:]:
        #print(p)
        k,v = p.split("=")
        ddic[k]=v.strip()
    if s_parts[0]=="#d":
        recieve_data(**ddic)
    
c_buff=[]
inbuff=False
while True:
    c = ser.read()
    if c==b"#":
        c_buff=[]
        inbuff=True
    if inbuff:
        if c==b"\n":
           # try:
            read_buff(c_buff)
           # except Exception as e:
           #     print(e)
           #     pass
            c_buff=[]
            inbuff=False
        else:
            c_buff.append(c)

KeyboardInterrupt: 

In [11]:
ser.close()
df

,light
2022-01-10 00:44:41.000,0
2022-01-10 00:44:41.400,128
2022-01-10 00:44:41.500,135
2022-01-10 00:44:41.600,250
2022-01-10 00:44:41.700,250
...,...
2022-01-10 00:45:16.200,232
2022-01-10 00:45:16.300,234
2022-01-10 00:45:16.400,235
2022-01-10 00:45:16.500,232
